# 导入需要的库

In [1]:
import numpy as np
import functions as f
from gradient import numerical_gradient
from collections import OrderedDict
from layers import *
import pickle
import struct
import matplotlib.pyplot as plt

# 定义读取数据集的函数

In [2]:

def load_mnist_data(kind):
    '''
    加载数据集
    :param kind: 加载训练数据还是测试数据
    :return: 打平之后的数据和one hot编码的标签
    '''
    labels_path = '../data/%s-labels-idx1-ubyte' % kind
    images_path = '../data/%s-images-idx3-ubyte' % kind
    with open(labels_path, 'rb') as lbpath:
        struct.unpack('>II', lbpath.read(8))
        labels = np.fromfile(lbpath, dtype=np.uint8)
    with open(images_path, 'rb') as imgpath:
        struct.unpack('>IIII', imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)

    return images / 255., np.eye(10)[labels]

# 定义神经网络架构

In [3]:

class NLayerMLP:

    def __init__(self, sizes, weight_init_method):
        # 存储权重值和偏置值
        self.sizes = sizes
        self.weights = []
        self.biases = []
        for input_size, output_size in zip(sizes[0:-1], sizes[1:]):
            # 初始化权重
            if weight_init_method=="0":
                # 使用0值初始化
                weight = 0 * np.random.randn(input_size, output_size)
            elif weight_init_method=="Kaiming":
                # 使用Kaiming初始化
                weight = 16*2/(input_size+output_size) * np.random.randn(input_size, output_size)
            bias = np.zeros(output_size)
            self.weights.append(weight)
            self.biases.append(bias)
        # 这个神经网络的层数为n-1
        size = len(sizes)-1
        # 生成层
        self.layers = OrderedDict()
        for i in range(0, size):
            if i != size-1:
                self.layers[f'Affine{i}'] = Affine(self.weights[i], self.biases[i])
                #self.layers['Relu1'] = Relu()
                self.layers[f'Sigmoid1{i}'] = Sigmoid()
            else:
                # 这里没有激活函数
                self.layers[f'Affine{i}'] = Affine(self.weights[i], self.biases[i])
        self.lastLayer = SoftmaxWithLoss()
        #self.lastLayer = IdentityWithLoss()
        self.params = dict()
        self.params['sizes'] = self.sizes
        self.params['biases'] = self.biases
        self.params['weights'] = self.weights
        self.params['layers'] = self.layers

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    # x:输入数据, t:监督数据
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # x:输入数据, t:监督数据
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        for i in range(0, len(self.sizes)-1):
            grads[f'W{i}'] = numerical_gradient(loss_W, self.weights[i])
            grads[f'b{i}'] = numerical_gradient(loss_W, self.biases[i])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 设定
        grads = {}
        for i in range(0, len(self.sizes)-1):
            grads[f'W{i}'], grads[f'b{i}'] = self.layers[f'Affine{i}'].dW, self.layers[f'Affine{i}'].db

        return grads



    def sgd(self, training_data, epochs, batch_size, lr, test_data=None, iter_num=10, save_path=None,
            loss_path=None):
        train_features, train_labels = training_data
        test_features, test_labels = test_data
        train_size = len(train_labels)
        test_size = len(test_labels)
        for i in range(epochs):
            for batch_index in range(0, train_size, batch_size):
                lower_range = batch_index
                upper_range = batch_index + batch_size
                if upper_range > train_size:
                    upper_range = train_size
                x_batch = train_features[lower_range: upper_range, :]
                y_batch = train_labels[lower_range: upper_range]
                # 计算梯度
                #grad = self.numerical_gradient(x_batch, y_batch)
                grad = self.gradient(x_batch, y_batch)
                # 更新参数
                for index in range(len(network.sizes)-1):
                    self.weights[index] -= lr * grad[f"W{index}"]
                    self.biases[index] -= lr * grad[f"b{index}"]

            loss = self.loss(x_batch, y_batch)
            loss_test = self.loss(test_images, test_labels)
            accuracy = np.sum(self.predict(test_features).argmax(1) == test_labels.argmax(1))/test_size
            #if i % iter_per_epoch == 0:
            if i % iter_num == 0:
                print(f"epoch: {i}, train loss, test loss , accuracy |{loss} , {loss_test}, {accuracy*100}%")
                if save_path:
                    self.save(save_path)
                    print("Weight has been saved !!")
                if loss_path:
                    with open(loss_path, "a") as f:
                        if i == 0:
                            f.write("epoch train_loss test_loss  accuracy\n")
                            f.write(f"{loss}  {loss_test} {accuracy}\n")
                        else:
                            f.write(f"{loss}  {loss_test} {accuracy}\n")


    def adam(self, training_data, epochs, batch_size, lr, test_data=None, iter_num=10, save_path=None,
             loss_path=None):
        train_features, train_labels = training_data
        test_features, test_labels = test_data
        train_size = len(train_labels)
        test_size = len(test_labels)
        for i in range(epochs):
            for batch_index in range(0, train_size, batch_size):
                lower_range = batch_index
                upper_range = batch_index + batch_size
                if upper_range > train_size:
                    upper_range = train_size
                x_batch = train_features[lower_range: upper_range, :]
                y_batch = train_labels[lower_range: upper_range]
                # 计算梯度
                #grad = self.numerical_gradient(x_batch, y_batch)
                grad = self.gradient(x_batch, y_batch)
                # 更新参数, 这里使用adam算法
                rho1, rho2 = 0.9, 0.999
                delta = 1e-8
                for index in range(len(network.sizes)-1):
                    weight = self.weights[index]
                    weight_grad = grad[f"W{index}"]
                    s = np.zeros_like(weight)
                    r = np.zeros_like(weight)
                    # update the first and the second moment
                    s = rho1 * s + (1 - rho1) * weight_grad
                    r = rho2 * r + (1 - rho2) * np.square(weight_grad)
                    # get the partial first and second moment
                    s_hat = s / (1 - rho1 ** (i + 1))
                    r_hat = r / (1 - rho1 ** (i + 1))
                    # use the partial first and second moment to modify the GD method
                    self.weights[index] -= lr * s_hat / (np.sqrt(r_hat) + delta)
                    bias = self.biases[index]
                    bias_grad = grad[f"b{index}"]
                    s = np.zeros_like(bias)
                    r = np.zeros_like(bias)
                    # update the first and the second moment
                    s = rho1 * s + (1 - rho1) * bias_grad
                    r = rho2 * r + (1 - rho2) * np.square(bias_grad)
                    # get the partial first and second moment
                    s_hat = s / (1 - rho1 ** (i + 1))
                    r_hat = r / (1 - rho1 ** (i + 1))
                    # use the partial first and second moment to modify the GD method
                    self.biases[index] -= lr * s_hat / (np.sqrt(r_hat) + delta)

            loss = self.loss(x_batch, y_batch)
            loss_test = self.loss(test_images, test_labels)
            accuracy = np.sum(self.predict(test_features).argmax(1) == test_labels.argmax(1))/test_size
            #if i % iter_per_epoch == 0:
            if i % iter_num == 0:
                print(f"epoch: {i}, train loss, test loss , accuracy |{loss} , {loss_test}, {accuracy*100}%")
                if save_path:
                    self.save(save_path)
                    print("Weight has been saved !!")
                if loss_path:
                    with open(loss_path, "a") as f:
                        if i == 0:
                            f.write("epoch train_loss test_loss  accuracy\n")
                            f.write(f"{i}  {loss}  {loss_test} {accuracy}\n")
                        else:
                            f.write(f"{i}  {loss} {loss_test} {accuracy}\n")

    # 保存权重文件
    def save(self,path):
        with open(path,'wb') as f :
            self.params['sizes'] = self.sizes
            self.params['biases'] = self.biases
            self.params['weights'] = self.weights
            self.params['layers'] = self.layers
            pickle.dump(self.params, f)

    # 导入权重文件
    def load(self,path):
        with open(path,'rb') as f :
            self.params = pickle.load(f)
            self.sizes = self.params['sizes']
            self.biases = self.params['biases']
            self.weights = self.params['weights']
            self.layers = self.params['layers']


        #print(weight)



# 导入训练集和测试集

In [4]:
train_images, train_labels = load_mnist_data(kind='train')
train_images = np.reshape(train_images, newshape=(-1, 1, 784))
test_images, test_labels = load_mnist_data('t10k')
test_images = np.reshape(test_images, newshape=(-1, 1, 784))

In [5]:
training_set_size = len(train_images)
testing_set_size = len(test_images)

In [6]:
# 定义神经网络架构
network = NLayerMLP(sizes=[784, 64, 32, 10])
weight_save_path  = "weight_multi_layer_adam.pickle"
epoches = 10  # 适当设定循环的次数
train_size = train_images.shape[0]
batch_size = 128
learning_rate = 1e-4
train_loss_list = []
test_loss_list = []
iter_num = 10
load_weight = True
train_net = True
if load_weight:
    network.load(weight_save_path)
if True:
    accuracy = np.sum(network.predict(test_images).argmax(1) == test_labels.argmax(1))/testing_set_size
    print(f"epoch: , train loss, test loss , accuracy |, {accuracy*100}%")
    print("Weight has been saved !!")
network.adam(training_data=(train_images, train_labels), epochs=epoches, batch_size=batch_size,
            lr=learning_rate, test_data=(test_images, test_labels), iter_num=1,
            save_path=weight_save_path)

epoch: , train loss, test loss , accuracy |, 95.69%
Weight has been saved !!
epoch: 0, train loss, test loss , accuracy |0.2064751866656389 , 0.18829753377274142, 95.57%
Weight has been saved !!
epoch: 1, train loss, test loss , accuracy |0.20866766375115844 , 0.18801436245030517, 95.62%
Weight has been saved !!
epoch: 2, train loss, test loss , accuracy |0.20935990598882162 , 0.18808573095869793, 95.65%
Weight has been saved !!
epoch: 3, train loss, test loss , accuracy |0.20933521848645575 , 0.1881304924603436, 95.65%
Weight has been saved !!
epoch: 4, train loss, test loss , accuracy |0.2091899965339419 , 0.18840798472866396, 95.69%
Weight has been saved !!
epoch: 5, train loss, test loss , accuracy |0.2090098357010374 , 0.18879363092310564, 95.67999999999999%
Weight has been saved !!
epoch: 6, train loss, test loss , accuracy |0.20881909358196996 , 0.18903879330508166, 95.66%
Weight has been saved !!
epoch: 7, train loss, test loss , accuracy |0.20864400239084865 , 0.18932176496038

# 验证权重和偏置都为0时

In [52]:
# 定义神经网络架构
network = NLayerMLP(sizes=[784, 128, 10])
weight_save_path  = "weight_64_hidden.pickle"
epoches = 10000  # 适当设定循环的次数
train_size = train_images.shape[0]
batch_size = 128
learning_rate = 1e-2
train_loss_list = []
test_loss_list = []
iter_num = 1
load_weight = False
train_net = True
if load_weight:
    network.load(weight_save_path)
if True:
    accuracy = np.sum(network.predict(test_images).argmax(1) == test_labels.argmax(1))/testing_set_size
    print(f"epoch: , train loss, test loss , accuracy |, {accuracy*100}%")
    print("Weight has been saved !!")
network.sgd(training_data=(train_images, train_labels), epochs=epoches, batch_size=batch_size,
             lr=learning_rate, test_data=(test_images, test_labels), iter_num=iter_num,
             save_path=weight_save_path, loss_path="single_layer_numpy_sgd.txt")

epoch: 661, train loss, test loss , accuracy |1.6799289539170992 , 1.5306884115579271, 38.75%
Weight has been saved !!
epoch: 662, train loss, test loss , accuracy |1.6799115628929446 , 1.53066593287219, 38.75%
Weight has been saved !!
epoch: 663, train loss, test loss , accuracy |1.6798941480787288 , 1.530643524834076, 38.75%
Weight has been saved !!


KeyboardInterrupt: 

In [49]:
# 定义神经网络架构
network = NLayerMLP(sizes=[784, 32,4, 10])
weight_save_path  = "weight_64_hidden.pickle"
epoches = 100000  # 适当设定循环的次数
train_size = train_images.shape[0]
batch_size = 128
learning_rate = 1e-4
train_loss_list = []
test_loss_list = []
iter_num = 1
load_weight = True
train_net = True
if load_weight:
    network.load(weight_save_path)
if True:
    accuracy = np.sum(network.predict(test_images).argmax(1) == test_labels.argmax(1))/testing_set_size
    print(f"epoch: , train loss, test loss , accuracy |, {accuracy*100}%")
    print("Weight has been saved !!")
network.sgd(training_data=(train_images, train_labels), epochs=epoches, batch_size=batch_size,
            lr=learning_rate, test_data=(test_images, test_labels), iter_num=iter_num,
            save_path=weight_save_path, loss_path="single_layer_numpy_adam4.txt")

epoch: , train loss, test loss , accuracy |, 20.75%
Weight has been saved !!
epoch: 0, train loss, test loss , accuracy |2.1283100007268407 , 2.076106760557496, 20.75%
Weight has been saved !!
epoch: 1, train loss, test loss , accuracy |2.1282113165164653 , 2.076112322932562, 20.75%
Weight has been saved !!
epoch: 2, train loss, test loss , accuracy |2.1281253676766254 , 2.0761171416420927, 20.74%
Weight has been saved !!
epoch: 3, train loss, test loss , accuracy |2.1280514256212837 , 2.0761211863424385, 20.74%
Weight has been saved !!
epoch: 4, train loss, test loss , accuracy |2.127988800356441 , 2.0761244284869242, 20.74%
Weight has been saved !!
epoch: 5, train loss, test loss , accuracy |2.1279368388675697 , 2.076126840473701, 20.73%
Weight has been saved !!
epoch: 6, train loss, test loss , accuracy |2.1278949235170113 , 2.07612839490973, 20.73%
Weight has been saved !!
epoch: 7, train loss, test loss , accuracy |2.1278624704614657 , 2.0761290640082795, 20.73%
Weight has been sa

KeyboardInterrupt: 

# 这一结用于研究不同的初始化对于最后训练得到的学习曲线的影响。

### 0权重初始化

In [8]:
# 定义神经网络架构
network = NLayerMLP(sizes=[784, 64, 32, 10], weight_init_method="0")
weight_save_path  = "weight_multi_layer_adam_0bias.pickle"
loss_path = "weight_multi_layer_adam_0bias.txt"
epoches = 500 # 适当设定循环的次数
train_size = train_images.shape[0]
batch_size = 128
learning_rate = 1e-4
load_weight = False
train_net = True
if load_weight:
    network.load(weight_save_path)
if True:
    accuracy = np.sum(network.predict(test_images).argmax(1) == test_labels.argmax(1))/testing_set_size
    print(f"epoch: , train loss, test loss , accuracy |, {accuracy*100}%")
    print("Weight has been saved !!")
network.adam(training_data=(train_images, train_labels), epochs=epoches, batch_size=batch_size,
            lr=learning_rate, test_data=(test_images, test_labels), iter_num=10,
            save_path=weight_save_path, loss_path=loss_path)

epoch: , train loss, test loss , accuracy |, 9.8%
Weight has been saved !!
epoch: 0, train loss, test loss , accuracy |2.154921193182849 , 2.1362275203968646, 10.54%
Weight has been saved !!
epoch: 10, train loss, test loss , accuracy |1.7680076426267544 , 1.7618344730282636, 22.400000000000002%
Weight has been saved !!
epoch: 20, train loss, test loss , accuracy |1.636665539048008 , 1.7196296295729327, 27.68%
Weight has been saved !!
epoch: 30, train loss, test loss , accuracy |1.6126512672328939 , 1.6387339668939715, 35.96%
Weight has been saved !!
epoch: 40, train loss, test loss , accuracy |1.5688362140343106 , 1.5916505507499537, 38.519999999999996%
Weight has been saved !!
epoch: 50, train loss, test loss , accuracy |1.535897347640858 , 1.570485058727916, 39.97%
Weight has been saved !!
epoch: 60, train loss, test loss , accuracy |1.5084495467610761 , 1.5618569318786477, 40.92%
Weight has been saved !!
epoch: 70, train loss, test loss , accuracy |1.4915745013996553 , 1.5571132590

KeyboardInterrupt: 

### 下面研究使用kaiming初始化的影响

In [11]:
# 定义神经网络架构,使用kaiming初始化
network = NLayerMLP(sizes=[784, 64, 32, 10], weight_init_method="Kaiming")
weight_save_path  = "weight_multi_layer_adam_Kaiming.pickle"
loss_path = "weight_multi_layer_adam_Kaiming.txt"
epoches = 500  # 适当设定循环的次数
train_size = train_images.shape[0]
batch_size = 128
learning_rate = 1e-4
load_weight = False
train_net = True
if load_weight:
    network.load(weight_save_path)
if True:
    accuracy = np.sum(network.predict(test_images).argmax(1) == test_labels.argmax(1))/testing_set_size
    print(f"epoch: , train loss, test loss , accuracy |, {accuracy*100}%")
    print("Weight has been saved !!")
network.adam(training_data=(train_images, train_labels), epochs=epoches, batch_size=batch_size,
            lr=learning_rate, test_data=(test_images, test_labels), iter_num=10,
            save_path=weight_save_path, loss_path=loss_path)

epoch: , train loss, test loss , accuracy |, 11.35%
Weight has been saved !!
epoch: 0, train loss, test loss , accuracy |0.3154867903619187 , 0.33321272605502655, 90.38000000000001%
Weight has been saved !!
epoch: 10, train loss, test loss , accuracy |0.23565516152895985 , 0.18183001901991053, 94.53%
Weight has been saved !!
epoch: 20, train loss, test loss , accuracy |0.23938485813044816 , 0.15948160241686202, 95.19999999999999%
Weight has been saved !!
epoch: 30, train loss, test loss , accuracy |0.23993347796150663 , 0.15057994938864555, 95.53%
Weight has been saved !!
epoch: 40, train loss, test loss , accuracy |0.23539109279314452 , 0.14583404336679237, 95.82000000000001%
Weight has been saved !!
epoch: 50, train loss, test loss , accuracy |0.22438219965303996 , 0.14363710572699034, 96.0%
Weight has been saved !!
epoch: 60, train loss, test loss , accuracy |0.2175692122833194 , 0.14449644705738077, 96.03%
Weight has been saved !!
epoch: 70, train loss, test loss , accuracy |0.2094

# 研究学习率调整策略对于输出的影响

## 固定学习率

In [13]:
# 定义神经网络架构,使用kaiming初始化
network = NLayerMLP(sizes=[784, 64, 32, 10], weight_init_method="Kaiming")
weight_save_path  = "weight_multi_layer_adam_fixed_lr.pickle"
loss_path = "weight_multi_layer_adam_fixed_lr.txt"
epoches = 100  # 适当设定循环的次数
train_size = train_images.shape[0]
batch_size = 128
learning_rate = 1e-4
load_weight = False
train_net = True
if load_weight:
    network.load(weight_save_path)
if True:
    accuracy = np.sum(network.predict(test_images).argmax(1) == test_labels.argmax(1))/testing_set_size
    print(f"epoch: , train loss, test loss , accuracy |, {accuracy*100}%")
    print("Weight has been saved !!")
network.adam(training_data=(train_images, train_labels), epochs=epoches, batch_size=batch_size,
            lr=learning_rate, test_data=(test_images, test_labels), iter_num=1,
            save_path=weight_save_path, loss_path=loss_path)

epoch: , train loss, test loss , accuracy |, 10.280000000000001%
Weight has been saved !!
epoch: 0, train loss, test loss , accuracy |0.36999324787357185 , 0.34525710337647275, 90.47%
Weight has been saved !!
epoch: 1, train loss, test loss , accuracy |0.3108812395775607 , 0.28147270278859315, 91.75999999999999%
Weight has been saved !!
epoch: 2, train loss, test loss , accuracy |0.2864337502089635 , 0.25483394205216703, 92.45%
Weight has been saved !!
epoch: 3, train loss, test loss , accuracy |0.27278712678115696 , 0.23779430631398304, 92.99%
Weight has been saved !!
epoch: 4, train loss, test loss , accuracy |0.2640487231560845 , 0.2252342962847426, 93.21000000000001%
Weight has been saved !!
epoch: 5, train loss, test loss , accuracy |0.2564452196223177 , 0.21527052864182072, 93.45%
Weight has been saved !!
epoch: 6, train loss, test loss , accuracy |0.24927962358163122 , 0.20705110328986628, 93.73%
Weight has been saved !!
epoch: 7, train loss, test loss , accuracy |0.242578343006

### 指数衰减

In [26]:

from math import gamma


class NLayerMLP:

    def __init__(self, sizes, weight_init_method):
        # 存储权重值和偏置值
        self.sizes = sizes
        self.weights = []
        self.biases = []
        for input_size, output_size in zip(sizes[0:-1], sizes[1:]):
            # 初始化权重
            if weight_init_method=="0":
                # 使用0值初始化
                weight = 0 * np.random.randn(input_size, output_size)
            elif weight_init_method=="Kaiming":
                # 使用Kaiming初始化
                weight = 16*2/(input_size+output_size) * np.random.randn(input_size, output_size)
            bias = np.zeros(output_size)
            self.weights.append(weight)
            self.biases.append(bias)
        # 这个神经网络的层数为n-1
        size = len(sizes)-1
        # 生成层
        self.layers = OrderedDict()
        for i in range(0, size):
            if i != size-1:
                self.layers[f'Affine{i}'] = Affine(self.weights[i], self.biases[i])
                #self.layers['Relu1'] = Relu()
                self.layers[f'Sigmoid1{i}'] = Sigmoid()
            else:
                # 这里没有激活函数
                self.layers[f'Affine{i}'] = Affine(self.weights[i], self.biases[i])
        self.lastLayer = SoftmaxWithLoss()
        #self.lastLayer = IdentityWithLoss()
        self.params = dict()
        self.params['sizes'] = self.sizes
        self.params['biases'] = self.biases
        self.params['weights'] = self.weights
        self.params['layers'] = self.layers

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    # x:输入数据, t:监督数据
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # x:输入数据, t:监督数据
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        for i in range(0, len(self.sizes)-1):
            grads[f'W{i}'] = numerical_gradient(loss_W, self.weights[i])
            grads[f'b{i}'] = numerical_gradient(loss_W, self.biases[i])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 设定
        grads = {}
        for i in range(0, len(self.sizes)-1):
            grads[f'W{i}'], grads[f'b{i}'] = self.layers[f'Affine{i}'].dW, self.layers[f'Affine{i}'].db

        return grads



    def sgd(self, training_data, epochs, batch_size, lr, test_data=None, iter_num=10, save_path=None,
            loss_path=None):
        train_features, train_labels = training_data
        test_features, test_labels = test_data
        train_size = len(train_labels)
        test_size = len(test_labels)
        for i in range(epochs):
            for batch_index in range(0, train_size, batch_size):
                lower_range = batch_index
                upper_range = batch_index + batch_size
                if upper_range > train_size:
                    upper_range = train_size
                x_batch = train_features[lower_range: upper_range, :]
                y_batch = train_labels[lower_range: upper_range]
                # 计算梯度
                #grad = self.numerical_gradient(x_batch, y_batch)
                grad = self.gradient(x_batch, y_batch)
                # 更新参数
                for index in range(len(network.sizes)-1):
                    self.weights[index] -= lr * grad[f"W{index}"]
                    self.biases[index] -= lr * grad[f"b{index}"]

            loss = self.loss(x_batch, y_batch)
            loss_test = self.loss(test_images, test_labels)
            accuracy = np.sum(self.predict(test_features).argmax(1) == test_labels.argmax(1))/test_size
            #if i % iter_per_epoch == 0:
            if i % iter_num == 0:
                print(f"epoch: {i}, train loss, test loss , accuracy |{loss} , {loss_test}, {accuracy*100}%")
                if save_path:
                    self.save(save_path)
                    print("Weight has been saved !!")
                if loss_path:
                    with open(loss_path, "a") as f:
                        if i == 0:
                            f.write("epoch train_loss test_loss  accuracy\n")
                            f.write(f"{loss}  {loss_test} {accuracy}\n")
                        else:
                            f.write(f"{loss}  {loss_test} {accuracy}\n")


    def adam(self, training_data, epochs, batch_size, lr, test_data=None, iter_num=10, save_path=None,
             loss_path=None):
        train_features, train_labels = training_data
        test_features, test_labels = test_data
        train_size = len(train_labels)
        test_size = len(test_labels)
        gamma = 0.999
        for i in range(epochs):
            for batch_index in range(0, train_size, batch_size):
                lower_range = batch_index
                upper_range = batch_index + batch_size
                if upper_range > train_size:
                    upper_range = train_size
                x_batch = train_features[lower_range: upper_range, :]
                y_batch = train_labels[lower_range: upper_range]
                # 计算梯度
                #grad = self.numerical_gradient(x_batch, y_batch)
                grad = self.gradient(x_batch, y_batch)
                # 更新参数, 这里使用adam算法
                rho1, rho2 = 0.9, 0.999
                delta = 1e-8
                for index in range(len(network.sizes)-1):
                    weight = self.weights[index]
                    weight_grad = grad[f"W{index}"]
                    s = np.zeros_like(weight)
                    r = np.zeros_like(weight)
                    # update the first and the second moment
                    s = rho1 * s + (1 - rho1) * weight_grad
                    r = rho2 * r + (1 - rho2) * np.square(weight_grad)
                    # get the partial first and second moment
                    s_hat = s / (1 - rho1 ** (i + 1))
                    r_hat = r / (1 - rho1 ** (i + 1))
                    # use the partial first and second moment to modify the GD method
                    self.weights[index] -= lr * s_hat / (np.sqrt(r_hat) + delta)
                    bias = self.biases[index]
                    bias_grad = grad[f"b{index}"]
                    s = np.zeros_like(bias)
                    r = np.zeros_like(bias)
                    # update the first and the second moment
                    s = rho1 * s + (1 - rho1) * bias_grad
                    r = rho2 * r + (1 - rho2) * np.square(bias_grad)
                    # get the partial first and second moment
                    s_hat = s / (1 - rho1 ** (i + 1))
                    r_hat = r / (1 - rho1 ** (i + 1))
                    # use the partial first and second moment to modify the GD method
                    self.biases[index] -= lr * s_hat / (np.sqrt(r_hat) + delta)
                lr *= gamma

            loss = self.loss(x_batch, y_batch)
            loss_test = self.loss(test_images, test_labels)
            accuracy = np.sum(self.predict(test_features).argmax(1) == test_labels.argmax(1))/test_size
            #if i % iter_per_epoch == 0:
            if i % iter_num == 0:
                print(f"epoch: {i}, train loss, test loss , accuracy |{loss} , {loss_test}, {accuracy*100}%")
                if save_path:
                    self.save(save_path)
                    print("Weight has been saved !!")
                if loss_path:
                    with open(loss_path, "a") as f:
                        if i == 0:
                            f.write("epoch train_loss test_loss  accuracy\n")
                            f.write(f"{i}  {loss}  {loss_test} {accuracy}\n")
                        else:
                            f.write(f"{i}  {loss} {loss_test} {accuracy}\n")

    # 保存权重文件
    def save(self,path):
        with open(path,'wb') as f :
            self.params['sizes'] = self.sizes
            self.params['biases'] = self.biases
            self.params['weights'] = self.weights
            self.params['layers'] = self.layers
            pickle.dump(self.params, f)

    # 导入权重文件
    def load(self,path):
        with open(path,'rb') as f :
            self.params = pickle.load(f)
            self.sizes = self.params['sizes']
            self.biases = self.params['biases']
            self.weights = self.params['weights']
            self.layers = self.params['layers']


        #print(weight)

# 定义神经网络架构,使用kaiming初始化
network = NLayerMLP(sizes=[784, 64, 32, 10], weight_init_method="Kaiming")
weight_save_path  = "weight_multi_layer_adam_e_lr.pickle"
loss_path = "weight_multi_layer_adam_e_lr.txt"
epoches = 100  # 适当设定循环的次数
train_size = train_images.shape[0]
batch_size = 128
learning_rate = 1e-2
load_weight = False
train_net = True
if load_weight:
    network.load(weight_save_path)
if True:
    accuracy = np.sum(network.predict(test_images).argmax(1) == test_labels.argmax(1))/testing_set_size
    print(f"epoch: , train loss, test loss , accuracy |, {accuracy*100}%")
    print("Weight has been saved !!")
network.adam(training_data=(train_images, train_labels), epochs=epoches, batch_size=batch_size,
            lr=learning_rate, test_data=(test_images, test_labels), iter_num=1,
            save_path=weight_save_path, loss_path=loss_path)

epoch: , train loss, test loss , accuracy |, 9.58%
Weight has been saved !!
epoch: 0, train loss, test loss , accuracy |0.19317891220512454 , 0.8017736782839289, 87.56%
Weight has been saved !!
epoch: 1, train loss, test loss , accuracy |0.1696701951360182 , 0.44602891171214043, 93.87%
Weight has been saved !!
epoch: 2, train loss, test loss , accuracy |0.16793174732309457 , 0.39880672253931093, 94.69999999999999%
Weight has been saved !!
epoch: 3, train loss, test loss , accuracy |0.16796456650479785 , 0.3610016610838291, 95.00999999999999%
Weight has been saved !!
epoch: 4, train loss, test loss , accuracy |0.1680064226485237 , 0.3539327935282079, 95.13000000000001%
Weight has been saved !!
epoch: 5, train loss, test loss , accuracy |0.16812004108932907 , 0.3482281300490547, 95.14%
Weight has been saved !!
epoch: 6, train loss, test loss , accuracy |0.1680170056098547 , 0.34886144257309354, 95.08%
Weight has been saved !!
epoch: 7, train loss, test loss , accuracy |0.1680185992995745

### 阶梯衰减

In [27]:
#
from math import gamma


class NLayerMLP:

    def __init__(self, sizes, weight_init_method):
        # 存储权重值和偏置值
        self.sizes = sizes
        self.weights = []
        self.biases = []
        for input_size, output_size in zip(sizes[0:-1], sizes[1:]):
            # 初始化权重
            if weight_init_method=="0":
                # 使用0值初始化
                weight = 0 * np.random.randn(input_size, output_size)
            elif weight_init_method=="Kaiming":
                # 使用Kaiming初始化
                weight = 16*2/(input_size+output_size) * np.random.randn(input_size, output_size)
            bias = np.zeros(output_size)
            self.weights.append(weight)
            self.biases.append(bias)
        # 这个神经网络的层数为n-1
        size = len(sizes)-1
        # 生成层
        self.layers = OrderedDict()
        for i in range(0, size):
            if i != size-1:
                self.layers[f'Affine{i}'] = Affine(self.weights[i], self.biases[i])
                #self.layers['Relu1'] = Relu()
                self.layers[f'Sigmoid1{i}'] = Sigmoid()
            else:
                # 这里没有激活函数
                self.layers[f'Affine{i}'] = Affine(self.weights[i], self.biases[i])
        self.lastLayer = SoftmaxWithLoss()
        #self.lastLayer = IdentityWithLoss()
        self.params = dict()
        self.params['sizes'] = self.sizes
        self.params['biases'] = self.biases
        self.params['weights'] = self.weights
        self.params['layers'] = self.layers

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    # x:输入数据, t:监督数据
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # x:输入数据, t:监督数据
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        for i in range(0, len(self.sizes)-1):
            grads[f'W{i}'] = numerical_gradient(loss_W, self.weights[i])
            grads[f'b{i}'] = numerical_gradient(loss_W, self.biases[i])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 设定
        grads = {}
        for i in range(0, len(self.sizes)-1):
            grads[f'W{i}'], grads[f'b{i}'] = self.layers[f'Affine{i}'].dW, self.layers[f'Affine{i}'].db

        return grads



    def sgd(self, training_data, epochs, batch_size, lr, test_data=None, iter_num=10, save_path=None,
            loss_path=None):
        train_features, train_labels = training_data
        test_features, test_labels = test_data
        train_size = len(train_labels)
        test_size = len(test_labels)
        for i in range(epochs):
            for batch_index in range(0, train_size, batch_size):
                lower_range = batch_index
                upper_range = batch_index + batch_size
                if upper_range > train_size:
                    upper_range = train_size
                x_batch = train_features[lower_range: upper_range, :]
                y_batch = train_labels[lower_range: upper_range]
                # 计算梯度
                #grad = self.numerical_gradient(x_batch, y_batch)
                grad = self.gradient(x_batch, y_batch)
                # 更新参数
                for index in range(len(network.sizes)-1):
                    self.weights[index] -= lr * grad[f"W{index}"]
                    self.biases[index] -= lr * grad[f"b{index}"]

            loss = self.loss(x_batch, y_batch)
            loss_test = self.loss(test_images, test_labels)
            accuracy = np.sum(self.predict(test_features).argmax(1) == test_labels.argmax(1))/test_size
            #if i % iter_per_epoch == 0:
            if i % iter_num == 0:
                print(f"epoch: {i}, train loss, test loss , accuracy |{loss} , {loss_test}, {accuracy*100}%")
                if save_path:
                    self.save(save_path)
                    print("Weight has been saved !!")
                if loss_path:
                    with open(loss_path, "a") as f:
                        if i == 0:
                            f.write("epoch train_loss test_loss  accuracy\n")
                            f.write(f"{loss}  {loss_test} {accuracy}\n")
                        else:
                            f.write(f"{loss}  {loss_test} {accuracy}\n")


    def adam(self, training_data, epochs, batch_size, lr, test_data=None, iter_num=10, save_path=None,
             loss_path=None):
        train_features, train_labels = training_data
        test_features, test_labels = test_data
        train_size = len(train_labels)
        test_size = len(test_labels)
        gamma = 0.999
        for i in range(epochs):
            for batch_index in range(0, train_size, batch_size):
                lower_range = batch_index
                upper_range = batch_index + batch_size
                if upper_range > train_size:
                    upper_range = train_size
                x_batch = train_features[lower_range: upper_range, :]
                y_batch = train_labels[lower_range: upper_range]
                # 计算梯度
                #grad = self.numerical_gradient(x_batch, y_batch)
                grad = self.gradient(x_batch, y_batch)
                # 更新参数, 这里使用adam算法
                rho1, rho2 = 0.9, 0.999
                delta = 1e-8
                for index in range(len(network.sizes)-1):
                    weight = self.weights[index]
                    weight_grad = grad[f"W{index}"]
                    s = np.zeros_like(weight)
                    r = np.zeros_like(weight)
                    # update the first and the second moment
                    s = rho1 * s + (1 - rho1) * weight_grad
                    r = rho2 * r + (1 - rho2) * np.square(weight_grad)
                    # get the partial first and second moment
                    s_hat = s / (1 - rho1 ** (i + 1))
                    r_hat = r / (1 - rho1 ** (i + 1))
                    # use the partial first and second moment to modify the GD method
                    self.weights[index] -= lr * s_hat / (np.sqrt(r_hat) + delta)
                    bias = self.biases[index]
                    bias_grad = grad[f"b{index}"]
                    s = np.zeros_like(bias)
                    r = np.zeros_like(bias)
                    # update the first and the second moment
                    s = rho1 * s + (1 - rho1) * bias_grad
                    r = rho2 * r + (1 - rho2) * np.square(bias_grad)
                    # get the partial first and second moment
                    s_hat = s / (1 - rho1 ** (i + 1))
                    r_hat = r / (1 - rho1 ** (i + 1))
                    # use the partial first and second moment to modify the GD method
                    self.biases[index] -= lr * s_hat / (np.sqrt(r_hat) + delta)
            if i%5==0:        
              lr *= gamma

            loss = self.loss(x_batch, y_batch)
            loss_test = self.loss(test_images, test_labels)
            accuracy = np.sum(self.predict(test_features).argmax(1) == test_labels.argmax(1))/test_size
            #if i % iter_per_epoch == 0:
            if i % iter_num == 0:
                print(f"epoch: {i}, train loss, test loss , accuracy |{loss} , {loss_test}, {accuracy*100}%")
                if save_path:
                    self.save(save_path)
                    print("Weight has been saved !!")
                if loss_path:
                    with open(loss_path, "a") as f:
                        if i == 0:
                            f.write("epoch train_loss test_loss  accuracy\n")
                            f.write(f"{i}  {loss}  {loss_test} {accuracy}\n")
                        else:
                            f.write(f"{i}  {loss} {loss_test} {accuracy}\n")

    # 保存权重文件
    def save(self,path):
        with open(path,'wb') as f :
            self.params['sizes'] = self.sizes
            self.params['biases'] = self.biases
            self.params['weights'] = self.weights
            self.params['layers'] = self.layers
            pickle.dump(self.params, f)

    # 导入权重文件
    def load(self,path):
        with open(path,'rb') as f :
            self.params = pickle.load(f)
            self.sizes = self.params['sizes']
            self.biases = self.params['biases']
            self.weights = self.params['weights']
            self.layers = self.params['layers']


        #print(weight)

# 定义神经网络架构,使用kaiming初始化
network = NLayerMLP(sizes=[784, 64, 32, 10], weight_init_method="Kaiming")
weight_save_path  = "weight_multi_layer_adam_step_lr.pickle"
loss_path = "weight_multi_layer_adam_step_lr.txt"
epoches = 100  # 适当设定循环的次数
train_size = train_images.shape[0]
batch_size = 128
learning_rate = 1e-2
load_weight = False
train_net = True
if load_weight:
    network.load(weight_save_path)
if True:
    accuracy = np.sum(network.predict(test_images).argmax(1) == test_labels.argmax(1))/testing_set_size
    print(f"epoch: , train loss, test loss , accuracy |, {accuracy*100}%")
    print("Weight has been saved !!")
network.adam(training_data=(train_images, train_labels), epochs=epoches, batch_size=batch_size,
            lr=learning_rate, test_data=(test_images, test_labels), iter_num=1,
            save_path=weight_save_path, loss_path=loss_path)

epoch: , train loss, test loss , accuracy |, 10.280000000000001%
Weight has been saved !!
epoch: 0, train loss, test loss , accuracy |0.3154399384451862 , 1.1588873630235947, 84.39999999999999%
Weight has been saved !!
epoch: 1, train loss, test loss , accuracy |0.17922283162186578 , 0.9391755776598456, 89.03999999999999%
Weight has been saved !!
epoch: 2, train loss, test loss , accuracy |0.18524941679578114 , 0.6520427899676913, 92.53%
Weight has been saved !!
epoch: 3, train loss, test loss , accuracy |0.2630809954264023 , 0.5947529770786859, 93.39%
Weight has been saved !!
epoch: 4, train loss, test loss , accuracy |0.17609907846694559 , 0.5744752285265365, 93.76%
Weight has been saved !!
epoch: 5, train loss, test loss , accuracy |0.20032665774703692 , 0.5320395659121544, 94.34%
Weight has been saved !!
epoch: 6, train loss, test loss , accuracy |0.37849415979606005 , 0.5368946255251452, 94.07%
Weight has been saved !!
epoch: 7, train loss, test loss , accuracy |0.2304809889065346

# 研究用时

### Adam算法，用时

In [16]:

import time

start = time.perf_counter()

# 定义神经网络架构,使用kaiming初始化
network = NLayerMLP(sizes=[784, 64, 32, 10], weight_init_method="Kaiming")
weight_save_path  = "adam_time_cost.pickle"
loss_path = "adam_time_cost.txt"
epoches = 100  # 适当设定循环的次数
train_size = train_images.shape[0]
batch_size = 128
learning_rate = 1e-4
load_weight = False
train_net = True
if load_weight:
    network.load(weight_save_path)
if True:
    accuracy = np.sum(network.predict(test_images).argmax(1) == test_labels.argmax(1))/testing_set_size
    print(f"epoch: , train loss, test loss , accuracy |, {accuracy*100}%")
    print("Weight has been saved !!")
network.adam(training_data=(train_images, train_labels), epochs=epoches, batch_size=batch_size,
             lr=learning_rate, test_data=(test_images, test_labels), iter_num=1,
             save_path=weight_save_path, loss_path=loss_path)
end = time.perf_counter()
print (f"用时{end-start}s")


epoch: , train loss, test loss , accuracy |, 9.8%
Weight has been saved !!
epoch: 0, train loss, test loss , accuracy |0.32255987506816347 , 0.34135853043388437, 90.31%
Weight has been saved !!
epoch: 1, train loss, test loss , accuracy |0.27601077467390817 , 0.2787177265647934, 91.73%
Weight has been saved !!
epoch: 2, train loss, test loss , accuracy |0.2571897467163539 , 0.25203624508936073, 92.43%
Weight has been saved !!
epoch: 3, train loss, test loss , accuracy |0.24714759983350706 , 0.23528934422578915, 92.91%
Weight has been saved !!
epoch: 4, train loss, test loss , accuracy |0.2424068423280071 , 0.22328609448262524, 93.28%
Weight has been saved !!
epoch: 5, train loss, test loss , accuracy |0.2382510212810832 , 0.21409585431809838, 93.62%
Weight has been saved !!
epoch: 6, train loss, test loss , accuracy |0.23473586222913964 , 0.206624964768192, 93.89%
Weight has been saved !!
epoch: 7, train loss, test loss , accuracy |0.231539375687523 , 0.20040290873070835, 94.07%
Weight

### 使用SGD算法用时

In [17]:

import time

start = time.perf_counter()

# 定义神经网络架构,使用kaiming初始化
network = NLayerMLP(sizes=[784, 64, 32, 10], weight_init_method="Kaiming")
weight_save_path  = "sgd_time_cost.pickle"
loss_path = "sgd_time_cost.txt"
epoches = 100  # 适当设定循环的次数
train_size = train_images.shape[0]
batch_size = 128
learning_rate = 1e-4
load_weight = False
train_net = True
if load_weight:
    network.load(weight_save_path)
if True:
    accuracy = np.sum(network.predict(test_images).argmax(1) == test_labels.argmax(1))/testing_set_size
    print(f"epoch: , train loss, test loss , accuracy |, {accuracy*100}%")
    print("Weight has been saved !!")
network.sgd(training_data=(train_images, train_labels), epochs=epoches, batch_size=batch_size,
             lr=learning_rate, test_data=(test_images, test_labels), iter_num=1,
             save_path=weight_save_path, loss_path=loss_path)
end = time.perf_counter()
print (f"用时{end-start}s")

epoch: , train loss, test loss , accuracy |, 10.280000000000001%
Weight has been saved !!
epoch: 0, train loss, test loss , accuracy |4.104411405115571 , 4.018607358761293, 10.280000000000001%
Weight has been saved !!
epoch: 1, train loss, test loss , accuracy |3.7158284245084077 , 3.639960437738529, 10.42%
Weight has been saved !!
epoch: 2, train loss, test loss , accuracy |3.4687640277202356 , 3.4002500146059798, 11.72%
Weight has been saved !!
epoch: 3, train loss, test loss , accuracy |3.2972626946474435 , 3.232428666476082, 12.76%
Weight has been saved !!
epoch: 4, train loss, test loss , accuracy |3.168391451849319 , 3.1046049792787365, 13.63%
Weight has been saved !!
epoch: 5, train loss, test loss , accuracy |3.064434316047716 , 2.9999241092365296, 14.92%
Weight has been saved !!
epoch: 6, train loss, test loss , accuracy |2.975801197741866 , 2.909481449605389, 17.19%
Weight has been saved !!
epoch: 7, train loss, test loss , accuracy |2.89737485988476 , 2.828703800651312, 20.2